# Desafio Kinea


## Importando

In [1]:
import logging
logging.level=logging.INFO

In [2]:
from typing import Dict, List
import pandas as pd

In [3]:
from ambima_connect import AmbimaConnect
import utils
from titulos import LTN, NTNB, NTNF, LFT

## Interface com API Ambima

In [19]:
def clean_list(list_to_clean:List[Dict], keep_keys):
  res = [{key : val for key, val in sub.items() if key in keep_keys} for sub in list_to_clean]
  return res

In [21]:
public = AmbimaConnect('titulos publicos').content
clean_public = clean_list(public, ['tipo_titulo','data_referencia','data_vencimento','taxa_indicativa','pu'])

In [5]:
priv = AmbimaConnect('titulos privados').content
priv_list = [x['codigo_ativo'] for x in priv]

## Cálculo Geral

In [ ]:
for titulo in clean_public[:10]:

  if 'tipo_titulo' in titulo:
    if titulo['tipo_titulo'] == 'LTN':
      calc = LTN(
        data_venc=titulo['data_vencimento'],
        taxa=titulo['taxa_indicativa']/100,
        data_ref=titulo['data_referencia'],
        )
    elif titulo['tipo_titulo'] == 'NTN-B':
      calc = NTNB(
        data_venc=titulo['data_vencimento'],
        taxa=titulo['taxa_indicativa']/100,
        data_ref=titulo['data_referencia'],
        )
    elif titulo['tipo_titulo'] == 'NTN-F':
      calc = NTNF(
        data_venc=titulo['data_vencimento'],
        taxa=titulo['taxa_indicativa']/100,
        data_ref=titulo['data_referencia'],
        )
    elif titulo['tipo_titulo'] == 'LFT':
      calc = LFT(
        data_venc=titulo['data_vencimento'],
        taxa=titulo['taxa_indicativa']/100,
        data_ref=titulo['data_referencia'],
        )
    
  titulo['pu_calc']=calc.pu
  titulo['pm_calc']=calc.pm
  titulo['duration_calc']=calc.duration

In [ ]:
t = clean_public[:10]
t_l = list()
for x in t: t_l.append({
  "Tipo": x['tipo_titulo'],
  "Data Ref": x['data_referencia'],
  "PU Real": x['pu'],
  "PU Calc": x['pu_calc'],
  "PU Error": utils.check_error(x['pu_calc'], x['pu']),
  # "Duration Real": x['duration'],
  "Duration Calc": x['duration_calc'],
  # "Duration Error": utils.check_error(x['duration_calc'], x['duration'])
  })
pd.DataFrame.from_dict(t_l)